# Population Density map of Hungary

In [1]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import folium
import re
import pydeck as pdk

## First we get some locations from a website
    We start by getting some counties because from then can we access the locations

In [2]:
all_counties=[]

url = 'https://www.citypopulation.de/en/hungary/'
data = requests.get(url)

html = BeautifulSoup(data.text, 'html.parser')

for a in html.find_all('a', href=True):
    all_counties.append(a['href'])
#https://www.citypopulation.de/en/hungary/
all_counties=all_counties[4:]
pprint(all_counties)

['bacskiskun/',
 'baranya/',
 'bekes/',
 'borsodabaujzemplen/',
 'budapestcity/',
 'csongrad/',
 'fejer/',
 'gyormosonsopron/',
 'hajdubihar/',
 'heves/',
 'jasznagykunszolnok/',
 'komaromesztergom/',
 'nograd/',
 'pest/',
 'somogy/',
 'szabolcsszatmarbereg/',
 'tolna/',
 'vas/',
 'veszprem/',
 'zala/']


## We get every city/village from every county and save it to a csv

In [3]:
have_titles=False
data = []
for county in all_counties:
    url = 'https://www.citypopulation.de/en/hungary/' + county
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    tables = html.find('table',id="ts")
    rows = tables.find_all('tr')
    
    if have_titles ==False:
        header_row = rows[0]
        columns = header_row.find_all('th')
        column_names = [col.get_text(strip=True) for col in columns]
    
    data_rows = rows[1:]
    
    
    for row in data_rows:
        row_data = [col.get_text(strip=True) for col in row.find_all('td')]
        data.append(row_data)
df = pd.DataFrame(data, columns=column_names)
display(df)
df.to_csv('Cities_pop.csv',index=False)

,Name,Status,District,PopulationCensus1980-01-01,PopulationCensus1990-01-01,PopulationCensus2001-02-01,PopulationCensus2011-10-01,PopulationEstimate2019-01-01,
0,Ágasegyháza,Village,Kecskemét,"2,110","1,956","1,916","1,893","1,889",→
1,Akasztó,Village,Kiskőrös,"3,983","3,664","3,519","3,354","3,221",→
2,Apostag,Village,Kunszentmiklós,"2,281","2,149","2,107","2,056","1,995",→
3,Bácsalmás,Town,Bácsalmás,"8,562","7,856","7,650","6,753","6,379",→
4,Bácsbokod,Large Village,Bácsalmás,"3,456","3,175","3,045","2,676","2,429",→
...,...,...,...,...,...,...,...,...,...
3173,Zalatárnok,Village,Zalaegerszeg,992,833,793,695,615,→
3174,Zalaújlak,Village,Nagykanizsa,253,174,129,101,90,→
3175,Zalavár,Village,Keszthely,"1,238","1,125",915,912,812,→
3176,Zalavég,Village,Zalaszentgrót,735,567,447,370,318,→


## We need to have the coordinates for these to put them on a map
    I have found an other website which lets me paste in any location and it gives me an address so this part automates that process

In [214]:
# URL of the webpage with the form
url = 'https://www.geonames.org/search.html?q='
data=[]
for count, city in enumerate(df.Name):

    print(city," [",count,"/ ",len(df.Name),"] ")
    url = 'https://www.geonames.org/search.html?q='+city+"&country=HU"
    # Send the GET request with the form data
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    tables = html.find_all('table')
    try:
        tables=tables[1]
        rows = tables.find_all('tr')
    
        data_rows = rows[2:3]
        for row in data_rows:
            row_data = [col.get_text(strip=True) for col in row.find_all('td')]
            row_data[1]=city
            row_data=row_data[1:]
            data.append(row_data)   
    except IndexError:
        print("This aint it chief")
        pass         
df2 = pd.DataFrame(data)
display(df2)
df2.to_csv('Cities_coord.csv',index=False)

Ágasegyháza  [ 0 /  3178 ] 
Akasztó  [ 1 /  3178 ] 
Apostag  [ 2 /  3178 ] 
Bácsalmás  [ 3 /  3178 ] 
Bácsbokod  [ 4 /  3178 ] 
Bácsborsód  [ 5 /  3178 ] 
Bácsszentgyörgy  [ 6 /  3178 ] 
Bácsszőlős  [ 7 /  3178 ] 
Baja  [ 8 /  3178 ] 
Ballószög  [ 9 /  3178 ] 
Balotaszállás  [ 10 /  3178 ] 
Bátmonostor  [ 11 /  3178 ] 
Bátya  [ 12 /  3178 ] 
Bócsa  [ 13 /  3178 ] 
Borota  [ 14 /  3178 ] 
Bugac  [ 15 /  3178 ] 
Bugacpusztaháza  [ 16 /  3178 ] 
Császártöltés  [ 17 /  3178 ] 
Csátalja  [ 18 /  3178 ] 
Csávoly  [ 19 /  3178 ] 
Csengőd  [ 20 /  3178 ] 
Csikéria  [ 21 /  3178 ] 
Csólyospálos  [ 22 /  3178 ] 
Dávod  [ 23 /  3178 ] 
Drágszél  [ 24 /  3178 ] 
Dunaegyháza  [ 25 /  3178 ] 
Dunafalva  [ 26 /  3178 ] 
Dunapataj  [ 27 /  3178 ] 
Dunaszentbenedek  [ 28 /  3178 ] 
Dunatetétlen  [ 29 /  3178 ] 
Dunavecse  [ 30 /  3178 ] 
Dusnok  [ 31 /  3178 ] 
Érsekcsanád  [ 32 /  3178 ] 
Érsekhalma  [ 33 /  3178 ] 
Fajsz  [ 34 /  3178 ] 
Felsőlajos  [ 35 /  3178 ] 
Felsőszentiván  [ 36 /  3178 ] 
Fok

Monyoród  [ 301 /  3178 ] 
Mozsgó  [ 302 /  3178 ] 
Nagybudmér  [ 303 /  3178 ] 
Nagycsány  [ 304 /  3178 ] 
Nagydobsza  [ 305 /  3178 ] 
Nagyhajmás  [ 306 /  3178 ] 
Nagyharsány  [ 307 /  3178 ] 
Nagykozár  [ 308 /  3178 ] 
Nagynyárád  [ 309 /  3178 ] 
Nagypall  [ 310 /  3178 ] 
Nagypeterd  [ 311 /  3178 ] 
Nagytótfalu  [ 312 /  3178 ] 
Nagyváty  [ 313 /  3178 ] 
Nemeske  [ 314 /  3178 ] 
Nyugotszenterzsébet  [ 315 /  3178 ] 
Óbánya  [ 316 /  3178 ] 
Ócsárd  [ 317 /  3178 ] 
Ófalu  [ 318 /  3178 ] 
Okorág  [ 319 /  3178 ] 
Okorvölgy  [ 320 /  3178 ] 
Olasz  [ 321 /  3178 ] 
Old  [ 322 /  3178 ] 
Orfű  [ 323 /  3178 ] 
Oroszló  [ 324 /  3178 ] 
Ózdfalu  [ 325 /  3178 ] 
Palé  [ 326 /  3178 ] 
Palkonya  [ 327 /  3178 ] 
Palotabozsok  [ 328 /  3178 ] 
Páprád  [ 329 /  3178 ] 
Patapoklosi  [ 330 /  3178 ] 
Pécs  [ 331 /  3178 ] 
Pécsbagota  [ 332 /  3178 ] 
Pécsdevecser  [ 333 /  3178 ] 
Pécsudvard  [ 334 /  3178 ] 
Pécsvárad  [ 335 /  3178 ] 
Pellérd  [ 336 /  3178 ] 
Pereked  [ 337 /  3

Galvács  [ 600 /  3178 ] 
Garadna  [ 601 /  3178 ] 
Gelej  [ 602 /  3178 ] 
Gesztely  [ 603 /  3178 ] 
Gibárt  [ 604 /  3178 ] 
Girincs  [ 605 /  3178 ] 
Golop  [ 606 /  3178 ] 
Gömörszőlős  [ 607 /  3178 ] 
Gönc  [ 608 /  3178 ] 
Göncruszka  [ 609 /  3178 ] 
Györgytarló  [ 610 /  3178 ] 
Halmaj  [ 611 /  3178 ] 
Hangács  [ 612 /  3178 ] 
Hangony  [ 613 /  3178 ] 
Háromhuta  [ 614 /  3178 ] 
Harsány  [ 615 /  3178 ] 
Hegymeg  [ 616 /  3178 ] 
Hejce  [ 617 /  3178 ] 
Hejőbába  [ 618 /  3178 ] 
Hejőkeresztúr  [ 619 /  3178 ] 
Hejőkürt  [ 620 /  3178 ] 
Hejőpapi  [ 621 /  3178 ] 
Hejőszalonta  [ 622 /  3178 ] 
Hercegkút  [ 623 /  3178 ] 
Hernádbűd  [ 624 /  3178 ] 
Hernádcéce  [ 625 /  3178 ] 
Hernádkak  [ 626 /  3178 ] 
Hernádkércs  [ 627 /  3178 ] 
Hernádnémeti  [ 628 /  3178 ] 
Hernádpetri  [ 629 /  3178 ] 
Hernádszentandrás  [ 630 /  3178 ] 
Hernádszurdok  [ 631 /  3178 ] 
Hernádvécse  [ 632 /  3178 ] 
Hét  [ 633 /  3178 ] 
Hidasnémeti  [ 634 /  3178 ] 
Hidvégardó  [ 635 /  3178 ] 
Ho

Bordány  [ 884 /  3178 ] 
Csanádalberti  [ 885 /  3178 ] 
Csanádpalota  [ 886 /  3178 ] 
Csanytelek  [ 887 /  3178 ] 
Csengele  [ 888 /  3178 ] 
Csongrád  [ 889 /  3178 ] 
Derekegyház  [ 890 /  3178 ] 
Deszk  [ 891 /  3178 ] 
Dóc  [ 892 /  3178 ] 
Domaszék  [ 893 /  3178 ] 
Eperjes  [ 894 /  3178 ] 
Fábiánsebestyén  [ 895 /  3178 ] 
Felgyő  [ 896 /  3178 ] 
Ferencszállás  [ 897 /  3178 ] 
Földeák  [ 898 /  3178 ] 
Forráskút  [ 899 /  3178 ] 
Hódmezővásárhely  [ 900 /  3178 ] 
Királyhegyes  [ 901 /  3178 ] 
Kistelek  [ 902 /  3178 ] 
Kiszombor  [ 903 /  3178 ] 
Klárafalva  [ 904 /  3178 ] 
Kövegy  [ 905 /  3178 ] 
Kübekháza  [ 906 /  3178 ] 
Magyarcsanád  [ 907 /  3178 ] 
Makó  [ 908 /  3178 ] 
Maroslele  [ 909 /  3178 ] 
Mártély  [ 910 /  3178 ] 
Mindszent  [ 911 /  3178 ] 
Mórahalom  [ 912 /  3178 ] 
Nagyér  [ 913 /  3178 ] 
Nagylak  [ 914 /  3178 ] 
Nagymágocs  [ 915 /  3178 ] 
Nagytőke  [ 916 /  3178 ] 
Óföldeák  [ 917 /  3178 ] 
Ópusztaszer  [ 918 /  3178 ] 
Öttömös  [ 919 /  3178 

Rábapordány  [ 1178 /  3178 ] 
Rábasebes  [ 1179 /  3178 ] 
Rábaszentandrás  [ 1180 /  3178 ] 
Rábaszentmihály  [ 1181 /  3178 ] 
Rábaszentmiklós  [ 1182 /  3178 ] 
Rábatamási  [ 1183 /  3178 ] 
Rábcakapi  [ 1184 /  3178 ] 
Rajka  [ 1185 /  3178 ] 
Ravazd  [ 1186 /  3178 ] 
Répceszemere  [ 1187 /  3178 ] 
Répcevis  [ 1188 /  3178 ] 
Rétalap  [ 1189 /  3178 ] 
Röjtökmuzsaj  [ 1190 /  3178 ] 
Románd  [ 1191 /  3178 ] 
Sarród  [ 1192 /  3178 ] 
Sikátor  [ 1193 /  3178 ] 
Sobor  [ 1194 /  3178 ] 
Sokorópátka  [ 1195 /  3178 ] 
Sopron  [ 1196 /  3178 ] 
Sopronhorpács  [ 1197 /  3178 ] 
Sopronkövesd  [ 1198 /  3178 ] 
Sopronnémeti  [ 1199 /  3178 ] 
Szakony  [ 1200 /  3178 ] 
Szany  [ 1201 /  3178 ] 
Szárföld  [ 1202 /  3178 ] 
Szerecseny  [ 1203 /  3178 ] 
Szil  [ 1204 /  3178 ] 
Szilsárkány  [ 1205 /  3178 ] 
Táp  [ 1206 /  3178 ] 
Tápszentmiklós  [ 1207 /  3178 ] 
Tarjánpuszta  [ 1208 /  3178 ] 
Tárnokréti  [ 1209 /  3178 ] 
Tényő  [ 1210 /  3178 ] 
Tét  [ 1211 /  3178 ] 
Töltéstava  [ 12

...

Orci  [ 2046 /  3178 ] 
Ordacsehi  [ 2047 /  3178 ] 
Öreglak  [ 2048 /  3178 ] 
Őrtilos  [ 2049 /  3178 ] 
Osztopán  [ 2050 /  3178 ] 
Ötvöskónyi  [ 2051 /  3178 ] 
Pálmajor  [ 2052 /  3178 ] 
Pamuk  [ 2053 /  3178 ] 
Patalom  [ 2054 /  3178 ] 
Patca  [ 2055 /  3178 ] 
Patosfa  [ 2056 /  3178 ] 
Péterhida  [ 2057 /  3178 ] 
Pogányszentpéter  [ 2058 /  3178 ] 
Polány  [ 2059 /  3178 ] 
Porrog  [ 2060 /  3178 ] 
Porrogszentkirály  [ 2061 /  3178 ] 
Porrogszentpál  [ 2062 /  3178 ] 
Potony  [ 2063 /  3178 ] 
Pusztakovácsi  [ 2064 /  3178 ] 
Pusztaszemes  [ 2065 /  3178 ] 
Ráksi  [ 2066 /  3178 ] 
Rinyabesenyő  [ 2067 /  3178 ] 
Rinyakovácsi  [ 2068 /  3178 ] 
Rinyaszentkirály  [ 2069 /  3178 ] 
Rinyaújlak  [ 2070 /  3178 ] 
Rinyaújnép  [ 2071 /  3178 ] 
Ságvár  [ 2072 /  3178 ] 
Sántos  [ 2073 /  3178 ] 
Sávoly  [ 2074 /  3178 ] 
Segesd  [ 2075 /  3178 ] 
Sérsekszőlős  [ 2076 /  3178 ] 
Simonfa  [ 2077 /  3178 ] 
Siófok  [ 2078 /  3178 ] 
Siójut  [ 2079 /  3178 ] 
Som  [ 2080 /  3178 ] 
S

Timár  [ 2334 /  3178 ] 
Tiszaadony  [ 2335 /  3178 ] 
Tiszabecs  [ 2336 /  3178 ] 
Tiszabercel  [ 2337 /  3178 ] 
Tiszabezdéd  [ 2338 /  3178 ] 
Tiszacsécse  [ 2339 /  3178 ] 
Tiszadada  [ 2340 /  3178 ] 
Tiszadob  [ 2341 /  3178 ] 
Tiszaeszlár  [ 2342 /  3178 ] 
Tiszakanyár  [ 2343 /  3178 ] 
Tiszakerecseny  [ 2344 /  3178 ] 
Tiszakóród  [ 2345 /  3178 ] 
Tiszalök  [ 2346 /  3178 ] 
Tiszamogyorós  [ 2347 /  3178 ] 
Tiszanagyfalu  [ 2348 /  3178 ] 
Tiszarád  [ 2349 /  3178 ] 
Tiszaszalka  [ 2350 /  3178 ] 
Tiszaszentmárton  [ 2351 /  3178 ] 
Tiszatelek  [ 2352 /  3178 ] 
Tiszavasvári  [ 2353 /  3178 ] 
Tiszavid  [ 2354 /  3178 ] 
Tisztaberek  [ 2355 /  3178 ] 
Tivadar  [ 2356 /  3178 ] 
Tornyospálca  [ 2357 /  3178 ] 
Tunyogmatolcs  [ 2358 /  3178 ] 
Túristvándi  [ 2359 /  3178 ] 
Túrricse  [ 2360 /  3178 ] 
Tuzsér  [ 2361 /  3178 ] 
Tyukod  [ 2362 /  3178 ] 
Újdombrád  [ 2363 /  3178 ] 
Újfehértó  [ 2364 /  3178 ] 
Újkenéz  [ 2365 /  3178 ] 
Ura  [ 2366 /  3178 ] 
Uszka  [ 2367 /  31

Orfalu  [ 2622 /  3178 ] 
Őrimagyarósd  [ 2623 /  3178 ] 
Őriszentpéter  [ 2624 /  3178 ] 
Ostffyasszonyfa  [ 2625 /  3178 ] 
Oszkó  [ 2626 /  3178 ] 
Pácsony  [ 2627 /  3178 ] 
Pankasz  [ 2628 /  3178 ] 
Pápoc  [ 2629 /  3178 ] 
Pecöl  [ 2630 /  3178 ] 
Perenye  [ 2631 /  3178 ] 
Peresznye  [ 2632 /  3178 ] 
Petőmihályfa  [ 2633 /  3178 ] 
Pinkamindszent  [ 2634 /  3178 ] 
Pornóapáti  [ 2635 /  3178 ] 
Porpác  [ 2636 /  3178 ] 
Pósfa  [ 2637 /  3178 ] 
Püspökmolnári  [ 2638 /  3178 ] 
Pusztacsó  [ 2639 /  3178 ] 
Rábagyarmat  [ 2640 /  3178 ] 
Rábahídvég  [ 2641 /  3178 ] 
Rábapaty  [ 2642 /  3178 ] 
Rábatöttös  [ 2643 /  3178 ] 
Rádóckölked  [ 2644 /  3178 ] 
Rátót  [ 2645 /  3178 ] 
Répcelak  [ 2646 /  3178 ] 
Répceszentgyörgy  [ 2647 /  3178 ] 
Rönök  [ 2648 /  3178 ] 
Rum  [ 2649 /  3178 ] 
Sajtoskál  [ 2650 /  3178 ] 
Salköveskút  [ 2651 /  3178 ] 
Sárfimizdó  [ 2652 /  3178 ] 
Sárvár  [ 2653 /  3178 ] 
Sé  [ 2654 /  3178 ] 
Simaság  [ 2655 /  3178 ] 
Sitke  [ 2656 /  3178 ] 
Söp

Veszprémgalsa  [ 2907 /  3178 ] 
Vid  [ 2908 /  3178 ] 
Vigántpetend  [ 2909 /  3178 ] 
Vilonya  [ 2910 /  3178 ] 
Vinár  [ 2911 /  3178 ] 
Vöröstó  [ 2912 /  3178 ] 
Zalaerdőd  [ 2913 /  3178 ] 
Zalagyömörő  [ 2914 /  3178 ] 
Zalahaláp  [ 2915 /  3178 ] 
Zalameggyes  [ 2916 /  3178 ] 
Zalaszegvár  [ 2917 /  3178 ] 
Zánka  [ 2918 /  3178 ] 
Zirc  [ 2919 /  3178 ] 
Alibánfa  [ 2920 /  3178 ] 
Almásháza  [ 2921 /  3178 ] 
Alsónemesapáti  [ 2922 /  3178 ] 
Alsópáhok  [ 2923 /  3178 ] 
Alsórajk  [ 2924 /  3178 ] 
Alsószenterzsébet  [ 2925 /  3178 ] 
Babosdöbréte  [ 2926 /  3178 ] 
Baglad  [ 2927 /  3178 ] 
Bagod  [ 2928 /  3178 ] 
Bak  [ 2929 /  3178 ] 
Baktüttös  [ 2930 /  3178 ] 
Balatongyörök  [ 2931 /  3178 ] 
Balatonmagyaród  [ 2932 /  3178 ] 
Bánokszentgyörgy  [ 2933 /  3178 ] 
Barlahida  [ 2934 /  3178 ] 
Batyk  [ 2935 /  3178 ] 
Bázakerettye  [ 2936 /  3178 ] 
Becsehely  [ 2937 /  3178 ] 
Becsvölgye  [ 2938 /  3178 ] 
Belezna  [ 2939 /  3178 ] 
Belsősárd  [ 2940 /  3178 ] 
Bezeréd 

,0,1,2,3,4
0,Ágasegyháza,"Hungary, Bács-Kiskun","populated placepopulation 1,898",N 46° 50' 24'',E 19° 27' 7''
1,Akasztó,"Hungary, Bács-Kiskun","populated placepopulation 3,480",N 46° 41' 30'',E 19° 12' 15''
2,Apostag,"Hungary, Bács-Kiskun","populated placepopulation 2,099",N 46° 52' 55'',E 18° 57' 43''
3,Bácsalmás,"Hungary, Bács-Kiskun",seat of a second-order administrative division...,N 46° 7' 35'',E 19° 19' 57''
4,Bácsbokod,"Hungary, Bács-Kiskun","populated placepopulation 3,077",N 46° 7' 30'',E 19° 9' 22''
...,...,...,...,...,...
3169,Zalatárnok,"Hungary, Zala",populated place,N 46° 42' 4'',E 16° 45' 34''
3170,Zalaújlak,"Hungary, Zala",populated place,N 46° 33' 35'',E 17° 4' 41''
3171,Zalavár,"Hungary, Zala",populated place,N 46° 40' 6'',E 17° 9' 21''
3172,Zalavég,"Hungary, Zala",populated place,N 47° 0' 24'',E 17° 1' 29''


## Next I merge these two csvs to have the coordinates and the populations together

  If you have the <b> Cities_coord.csv </b> and the <b> Cities_pop.csv </b> you can run the code from here after the necessary imports so you save quite some time

In [3]:
# Load the coordinate CSV
coordinates_df = pd.read_csv('Cities_coord.csv')
# Load the population CSV
population_df = pd.read_csv('Cities_pop.csv')
population_df = population_df.drop('Unnamed: 8', axis=1)
        

# Merge the two dataframes based on the common column (e.g., District)
df = pd.merge(coordinates_df, population_df, on='Name')

## I had some errors with the districts in Budapest where the coloumns were shifted
    This thing fixes it

In [4]:
for count, city in enumerate(df.Name):
    if "Budapest" in city:
        df.loc[df['Name'] == city,'PopulationEstimate2019-01-01'] = df.loc[df['Name'] == city,'PopulationCensus2011-10-01']
        df.loc[df['Name'] == city,'PopulationCensus2011-10-01'] = df.loc[df['Name'] == city,'PopulationCensus2001-02-01']
        df.loc[df['Name'] == city,'PopulationCensus2001-02-01'] = df.loc[df['Name'] == city,'PopulationCensus1990-01-01']
        df.loc[df['Name'] == city,'PopulationCensus1990-01-01'] = df.loc[df['Name'] == city,'PopulationCensus1980-01-01']
        df.loc[df['Name'] == city,'District_y'] = "Budapest"
df.to_csv('Cities.csv',index=False)
display(df)

,Name,District_x,Population,Latitude,Longitude,Status,District_y,PopulationCensus1980-01-01,PopulationCensus1990-01-01,PopulationCensus2001-02-01,PopulationCensus2011-10-01,PopulationEstimate2019-01-01
0,Ágasegyháza,"Hungary, Bács-Kiskun","populated placepopulation 1,898",N 46° 50' 24'',E 19° 27' 7'',Village,Kecskemét,"2,110","1,956","1,916","1,893","1,889"
1,Akasztó,"Hungary, Bács-Kiskun","populated placepopulation 3,480",N 46° 41' 30'',E 19° 12' 15'',Village,Kiskőrös,"3,983","3,664","3,519","3,354","3,221"
2,Apostag,"Hungary, Bács-Kiskun","populated placepopulation 2,099",N 46° 52' 55'',E 18° 57' 43'',Village,Kunszentmiklós,"2,281","2,149","2,107","2,056","1,995"
3,Bácsalmás,"Hungary, Bács-Kiskun",seat of a second-order administrative division...,N 46° 7' 35'',E 19° 19' 57'',Town,Bácsalmás,"8,562","7,856","7,650","6,753","6,379"
4,Bácsbokod,"Hungary, Bács-Kiskun","populated placepopulation 3,077",N 46° 7' 30'',E 19° 9' 22'',Large Village,Bácsalmás,"3,456","3,175","3,045","2,676","2,429"
...,...,...,...,...,...,...,...,...,...,...,...,...
3169,Zalatárnok,"Hungary, Zala",populated place,N 46° 42' 4'',E 16° 45' 34'',Village,Zalaegerszeg,992,833,793,695,615
3170,Zalaújlak,"Hungary, Zala",populated place,N 46° 33' 35'',E 17° 4' 41'',Village,Nagykanizsa,253,174,129,101,90
3171,Zalavár,"Hungary, Zala",populated place,N 46° 40' 6'',E 17° 9' 21'',Village,Keszthely,"1,238","1,125",915,912,812
3172,Zalavég,"Hungary, Zala",populated place,N 47° 0' 24'',E 17° 1' 29'',Village,Zalaszentgrót,735,567,447,370,318


## The coordinates are not in hte right format yet so they have to be converted

In [5]:
def convert_coordinates(coord_str):
    direction = coord_str[0]  # Extract the direction (N, S, E, W)
    degrees, minutes, seconds = map(int, re.findall(r'\d+', coord_str))  # Extract the numeric values

    decimal_degrees = degrees + minutes / 60 + seconds / 3600

    if direction in ['S', 'W']:
        decimal_degrees *= -1

    return decimal_degrees

## The following script does the necessary preprocessing and the displaying
    There were some NaN values in the csv which had to be filtered. Also the Budapest districts' population values were     int64 after conversion which throws an error to the displaying function so another casting is done right after the first one. Also I had to assign the Population values to an 'elevation' coloumn because it did not like the format of the first one.

In [6]:
df = pd.read_csv('Cities.csv')
df = df[['Name','Latitude','Longitude','PopulationEstimate2019-01-01']]
df.columns = ['Name','Latitude','Longitude','PopulationEstimate2019-01-01']
# Convert population values to float, handling empty strings
df = df.dropna()
df['Latitude'] = df['Latitude'].apply(convert_coordinates)
df['Longitude'] = df['Longitude'].apply(convert_coordinates)

# Convert population values to float, handling empty strings
df['PopulationEstimate2019-01-01'] = df['PopulationEstimate2019-01-01'].apply(lambda x: int(x.split()[0].replace(',', '')) if x and len(x.split()) > 0 else None)
# Drop rows with missing values
df['PopulationEstimate2019-01-01']=df['PopulationEstimate2019-01-01'].astype(float)

df['elevation'] = df['PopulationEstimate2019-01-01']
# Define a layer to display on a map
layer = pdk.Layer(
    "ColumnLayer",
    df,
    get_position=["Longitude", "Latitude"],
    elevation_scale=3,
    pickable=True,
    elevation_range=[0, df['elevation'].max()],
    extruded=True,
    coverage=0.5,
    radius=1500,
    stroked=False,
    filled=True,
    wireframe=True,
    get_fill_color=['elevation/10', 'elevation', 0, 230],
    get_elevation = 'elevation'
)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=19.0402,
    latitude=47.4979,
    zoom=7,
    pitch=30,
    bearing=0
)


tooltip = {"html": "<b>Name:</b> {Name} <br /><b>Population:</b> {elevation}",
    "style": {"background": "grey", "color": "white", "font-family": '"Helvetica Neue", Arial', "z-index": "10000"},}
# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state,tooltip=tooltip)
r.to_html("Population_density.html")